# Data modeling

In this notebook, we will do the modeling phase of predicting the traffic of cyclists crossing Nygåardsbroen. Using the prepared datasets from the data preparation steps, we will train, validate, and evaluate various regression algorithms, from simple linear models to more complex ones like Support Vector Regression. Throughout, our objective remains to minimize the root mean square error (RMSE) to ensure our predictions are as accurate as possible.

The data used is from: [Statens vegvesen](https://trafikkdata.atlas.vegvesen.no/) and [Geofysisk institutt](https://veret.gfi.uib.no/).

## Table of Contents
1. [Data Preparation](#Data-Preparation)
2. [Implementing the Baseline Model](#Implementing-the-Baseline-Model)
3. [Models](#Models)
    - [KNN](#KNN)
    - [Decision Tree](#Decision-Tree)
    - [Ridge and Lasso](#Ridge-and-Lasso)
    - [Polynomial Regression](#Polynomial-Regression)
    - [Support Vector Regression (SVR)](#Support-Vector-Regression-(SVR))
4. [Evaluating the Best Model](#Evaluating-the-Best-Model)
5. [Prediction](#Prediction)
6. [Visualize The Predicted Data](#Visualize-The-Predicted-Data)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as ex
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import pickle
from sklearn.linear_model import Ridge, Lasso
import random

In [2]:
SEED = 44
random.seed(44)
np.random.seed(44)


# Data Preparation
Loading the datasets

In [3]:
training_df = pd.read_csv('training_df.csv')
val_df  = pd.read_csv('val_df.csv')
test_df = pd.read_csv('test_df.csv')

In [4]:
training_df.head()

,dato_tid,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Lufttrykk,Trafikkmengde
0,2015-07-16 16:00:00,8.116667,13.733333,317.500000,4.333333,1014.400000,66
1,2015-07-16 17:00:00,10.000000,13.866667,318.166667,3.933333,1014.066667,44
2,2015-07-16 18:00:00,10.000000,13.216667,319.833333,4.233333,1013.966667,39
3,2015-07-16 19:00:00,10.000000,12.683333,323.500000,2.950000,1014.100000,29
4,2015-07-16 20:00:00,6.000000,12.066667,333.500000,2.483333,1014.200000,30


Checking if any of the data sets contains missing values

In [5]:
training_df.isna().any()

dato_tid          False
Solskinstid       False
Lufttemperatur    False
Vindretning       False
Vindstyrke        False
Lufttrykk         False
Trafikkmengde     False
dtype: bool

In [6]:
val_df.isna().any()

dato_tid          False
Solskinstid       False
Lufttemperatur    False
Vindretning       False
Vindstyrke        False
Lufttrykk         False
Trafikkmengde     False
dtype: bool

In [7]:
test_df.isna().any()

dato_tid          False
Solskinstid       False
Lufttemperatur    False
Vindretning       False
Vindstyrke        False
Lufttrykk         False
Trafikkmengde     False
dtype: bool

In [8]:
print(training_df['dato_tid'].dtype)
print(val_df['dato_tid'].dtype)
print(test_df['dato_tid'].dtype)

print(training_df['Trafikkmengde'].dtype)
print(val_df['Trafikkmengde'].dtype)
print(test_df['Trafikkmengde'].dtype)

object
object
object
int64
int64
int64


In [9]:
training_df['dato_tid'] = pd.to_datetime(training_df['dato_tid'])
val_df['dato_tid'] = pd.to_datetime(val_df['dato_tid'])
test_df['dato_tid'] = pd.to_datetime(test_df['dato_tid'])

<a id="subheading1_2"></a>
## Adding feature labels

In [10]:
for df in [training_df, val_df, test_df]:
    df['Hour'] = df['dato_tid'].dt.hour
    df['Day'] = df['dato_tid'].dt.day
    df['Month'] = df['dato_tid'].dt.month
    df['Year'] = df['dato_tid'].dt.year
    
# Drop the 'dato_tid' column
training_df.drop(columns=['dato_tid'], inplace=True)
val_df.drop(columns=['dato_tid'], inplace=True)
test_df.drop(columns=['dato_tid'], inplace=True)

In [11]:
training_df.head()

,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Lufttrykk,Trafikkmengde,Hour,Day,Month,Year
0,8.116667,13.733333,317.500000,4.333333,1014.400000,66,16,16,7,2015
1,10.000000,13.866667,318.166667,3.933333,1014.066667,44,17,16,7,2015
2,10.000000,13.216667,319.833333,4.233333,1013.966667,39,18,16,7,2015
3,10.000000,12.683333,323.500000,2.950000,1014.100000,29,19,16,7,2015
4,6.000000,12.066667,333.500000,2.483333,1014.200000,30,20,16,7,2015


**Splitting the data**

Spliting the data into X (features) and y (target) values

In [12]:
X_train = training_df.drop(columns=['Trafikkmengde'])
y_train = training_df['Trafikkmengde']
X_val = val_df.drop(columns=['Trafikkmengde'])
y_val = val_df['Trafikkmengde']

Scaling the feature data.

In [13]:
standardscaler = StandardScaler()
X_train_scaled = standardscaler.fit_transform(X_train)
X_val_scaled = standardscaler.transform(X_val)

# Implementing the Baseline Model

The purpose of the dummy regressor is to provide a naive baseline for comparison. We do not use the scaled fetures here.

In [14]:
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_train, y_train)

# Predict on the validation set
y_pred_val_dummy = dummy.predict(X_val)

# Computing RMSE for the dummy regressor
rmse_dummy = np.sqrt(mean_squared_error(y_val, y_pred_val_dummy))
print(f"Baseline RMSE (Dummy Regressor): {rmse_dummy:.2f}")

Baseline RMSE (Dummy Regressor): 39.77


# Models

**Training and evaluating the models**

# KNN

In [15]:
# Hyperparameter tuning for KNN
k_values = [1, 3, 5, 7, 9, 11, 13, 15]
best_rmse_knn = float('inf')
best_k = None

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred_val = knn.predict(X_val_scaled)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    if rmse < best_rmse_knn:
        best_rmse_knn = rmse
        best_k = k
    print(f"RMSE (K={k}): {rmse:.2f}")

print(f"Best RMSE for KNN: {best_rmse_knn:.2f} with K={best_k}")

RMSE (K=1): 44.22
RMSE (K=3): 38.77
RMSE (K=5): 37.71
RMSE (K=7): 37.10
RMSE (K=9): 36.96
RMSE (K=11): 36.81
RMSE (K=13): 36.65
RMSE (K=15): 36.55
Best RMSE for KNN: 36.55 with K=15


# Decision Tree

In [16]:
# Hyperparameter tuning for Decision Tree
depth_values = [None, 3, 5, 7, 9, 11]
best_rmse_dt = float('inf')
best_depth = None

for depth in depth_values:
    dt = DecisionTreeRegressor(max_depth=depth, random_state=SEED)
    dt.fit(X_train_scaled, y_train)
    y_pred_val = dt.predict(X_val_scaled)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    if rmse < best_rmse_dt:
        best_rmse_dt = rmse
        best_depth = depth
    print(f"RMSE (Max Depth={depth}): {rmse:.2f}")

print(f"Best RMSE for Decision Tree: {best_rmse_dt:.2f} with Max Depth={best_depth}")

RMSE (Max Depth=None): 36.21
RMSE (Max Depth=3): 38.41
RMSE (Max Depth=5): 30.29
RMSE (Max Depth=7): 29.21
RMSE (Max Depth=9): 26.97
RMSE (Max Depth=11): 28.66
Best RMSE for Decision Tree: 26.97 with Max Depth=9


# Ridge and Lasso

In [17]:
alphas = [0.01, 0.1, 1, 10, 100]

best_rmse = float('inf')
best_model_name = None
best_alpha = None

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    lasso = Lasso(alpha=alpha)
    for model, name in [(ridge, 'Ridge'), (lasso, 'Lasso')]:
        model.fit(X_train_scaled, y_train)
        y_pred_val = model.predict(X_val_scaled)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
        print(f"RMSE ({name}, Alpha={alpha}): {rmse:.2f}")

        # Update best RMSE
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_name = name
            best_alpha = alpha

print(f"\nBest RMSE obtained by {best_model_name} with Alpha={best_alpha}: {best_rmse:.2f}")

RMSE (Ridge, Alpha=0.01): 38.94
RMSE (Lasso, Alpha=0.01): 38.94
RMSE (Ridge, Alpha=0.1): 38.94
RMSE (Lasso, Alpha=0.1): 38.93
RMSE (Ridge, Alpha=1): 38.94
RMSE (Lasso, Alpha=1): 38.84
RMSE (Ridge, Alpha=10): 38.94
RMSE (Lasso, Alpha=10): 39.57
RMSE (Ridge, Alpha=100): 38.94
RMSE (Lasso, Alpha=100): 39.77

Best RMSE obtained by Lasso with Alpha=1: 38.84


# Polynomial Regression

In [18]:
degrees = [2, 3, 4]
best_rmse_poly = float('inf')
best_degree = None

for degree in degrees:
    poly_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    poly_model.fit(X_train_scaled, y_train)
    y_pred_val = poly_model.predict(X_val_scaled)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    if rmse < best_rmse_poly:
        best_rmse_poly = rmse
        best_degree = degree
    print(f"RMSE (Degree={degree}): {rmse:.2f}")

print(f"Best RMSE for Polynomial Regression: {best_rmse_poly:.2f} with Degree={best_degree}")

RMSE (Degree=2): 36.01
RMSE (Degree=3): 37.93
RMSE (Degree=4): 43.34
Best RMSE for Polynomial Regression: 36.01 with Degree=2


# Support Vector Regression (SVR)

In [19]:
C_values = [1, 10]
epsilon_values = [0.1, 1]
gamma_values = ['scale', 'auto']

best_rmse_svr = float('inf')
best_c = None
best_epsilon = None
best_gamma = None

for c in C_values:
    for epsilon in epsilon_values:
        for gamma in gamma_values:
            svr = SVR(kernel='rbf', C=c, epsilon=epsilon, gamma=gamma)
            svr.fit(X_train_scaled, y_train)
            y_pred_val = svr.predict(X_val_scaled)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
            if rmse < best_rmse_svr:
                best_rmse_svr = rmse
                best_c = c
                best_epsilon = epsilon
                best_gamma = gamma
            print(f"RMSE (C={c}, Epsilon={epsilon}, Gamma={gamma}): {rmse:.2f}")

print(f"Best RMSE for SVR: {best_rmse_svr:.2f} with C={best_c}, Epsilon={best_epsilon}, and Gamma={best_gamma}")

RMSE (C=1, Epsilon=0.1, Gamma=scale): 37.89
RMSE (C=1, Epsilon=0.1, Gamma=auto): 37.89
RMSE (C=1, Epsilon=1, Gamma=scale): 37.81
RMSE (C=1, Epsilon=1, Gamma=auto): 37.81
RMSE (C=10, Epsilon=0.1, Gamma=scale): 37.53
RMSE (C=10, Epsilon=0.1, Gamma=auto): 37.53
RMSE (C=10, Epsilon=1, Gamma=scale): 37.49
RMSE (C=10, Epsilon=1, Gamma=auto): 37.49
Best RMSE for SVR: 37.49 with C=10, Epsilon=1, and Gamma=scale


# Evaluating the Best Model

We found that the model with the lowest RMSE was the **decision tree** with a max depth of 9.

RMSE for Decision Tree: 26.97 with Max Depth=9

Before finalizing the decision on what model to use based on the validation set results, we will evaluate the best model's performance on the test set. This will give a more accurate measure of the model's ability to generalize to new, unseen data.

In [20]:
dt_best = DecisionTreeRegressor(max_depth=11, random_state=SEED)
dt_best.fit(X_train_scaled, y_train)
y_pred_test = dt_best.predict(standardscaler.transform(test_df.drop(columns=['Trafikkmengde'])))
rmse_test = np.sqrt(mean_squared_error(test_df['Trafikkmengde'], y_pred_test))
print(f"Test RMSE for Decision Tree: {rmse_test:.2f}")

Test RMSE for Decision Tree: 29.65


The test RMSE is slightly higher than the validation RMSE (29.65 vs 26.97). This is expected as the model was not trained on the test set, and it indicates that the model's performance on unseen data is consistent with the validation set.

In [21]:
# Get feature importance from the decision tree
feature_importance = dt_best.feature_importances_

# Map importance values with feature names
feature_importance_df = pd.DataFrame({
    'Feature': test_df.drop(columns=['Trafikkmengde']).columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)

          Feature  Importance
5            Hour    0.505187
8            Year    0.167734
7           Month    0.114410
1  Lufttemperatur    0.106394
0     Solskinstid    0.031132
4       Lufttrykk    0.029160
6             Day    0.022010
3      Vindstyrke    0.016169
2     Vindretning    0.007804


The output displays the importance of each feature when making predictions with the decision tree model dt_best. The importance is a value between 0 and 1, with higher values indicating that a feature has a stronger influence on the prediction. From the given output:

- **Hour** is the most influential feature, with an importance of approximately 0.5395. This suggests that the time of day plays a significant role in predicting 'Trafikkmengde'.
- **Year** is the next important feature with an importance of about 0.1908, followed by **Lufttemperatur** and **Month**, indicating that these variables also have substantial effects.
- Features like **Vindretning** and **Vindstyrke** have relatively low importance, suggesting they have minimal impact on the model's predictions.

From our data exploration, we identified that air temperature ('**Lufttemperatur**') and sunshine hours ('**Solskinstid**') both have strong positive correlations with bike traffic. Interestingly, our model's feature importance reflects a similar story. The feature Hour, which holds the highest importance, can be indicative of daily cycles in temperature and sunlight. This is because both air temperature and sunshine hours generally exhibit variations depending on the time of day. Additionally, the Year feature's high importance could be capturing annual patterns and changes in temperature and sunlight conditions, which again would influence bike traffic. In essence, while 'Lufttemperatur' and 'Solskinstid' individually show substantial importance, the dominant roles of Hour and Year in our model might be capturing the inherent patterns and correlations tied to these meteorological conditions and their impact on bike traffic

# Prediction
As the decision tree with the depth of 9 seemed to perform the best, we will select this model to do the prediction.

In [22]:
# Load the traffic_weather_df_2023.csv
predict_df = pd.read_csv('traffic_weather_df_2023.csv')

In [23]:
predict_df.head()

,Unnamed: 0,dato_tid,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Lufttrykk
0,0,2023-01-01 00:00:00,0.0,0.840000,42.800000,1.480000,994.840000
1,1,2023-01-01 01:00:00,0.0,0.583333,54.666667,1.283333,995.033333
2,2,2023-01-01 02:00:00,0.0,-0.200000,38.666667,0.733333,995.400000
3,3,2023-01-01 03:00:00,0.0,-1.316667,135.000000,0.950000,995.666667
4,4,2023-01-01 04:00:00,0.0,0.833333,64.333333,3.266667,995.566667


In [24]:
training_df.head()

,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Lufttrykk,Trafikkmengde,Hour,Day,Month,Year
0,8.116667,13.733333,317.500000,4.333333,1014.400000,66,16,16,7,2015
1,10.000000,13.866667,318.166667,3.933333,1014.066667,44,17,16,7,2015
2,10.000000,13.216667,319.833333,4.233333,1013.966667,39,18,16,7,2015
3,10.000000,12.683333,323.500000,2.950000,1014.100000,29,19,16,7,2015
4,6.000000,12.066667,333.500000,2.483333,1014.200000,30,20,16,7,2015


In [25]:
# Convert dato_tid to datetime and extract hour, day, month, and year
predict_df['dato_tid'] = pd.to_datetime(predict_df['dato_tid'])
predict_df['Hour'] = predict_df['dato_tid'].dt.hour
predict_df['Day'] = predict_df['dato_tid'].dt.day
predict_df['Month'] = predict_df['dato_tid'].dt.month
predict_df['Year'] = predict_df['dato_tid'].dt.year

predict_df.drop(columns=['dato_tid'], inplace=True)
predict_df.drop(columns=['Unnamed: 0'], inplace=True)
predict_df.head()

,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Lufttrykk,Hour,Day,Month,Year
0,0.0,0.840000,42.800000,1.480000,994.840000,0,1,1,2023
1,0.0,0.583333,54.666667,1.283333,995.033333,1,1,1,2023
2,0.0,-0.200000,38.666667,0.733333,995.400000,2,1,1,2023
3,0.0,-1.316667,135.000000,0.950000,995.666667,3,1,1,2023
4,0.0,0.833333,64.333333,3.266667,995.566667,4,1,1,2023


In [26]:
X_predict_scaled = standardscaler.transform(predict_df)

dt_best = DecisionTreeRegressor(max_depth=9, random_state=SEED)
dt_best.fit(X_train_scaled, y_train)

predictions_2023 = dt_best.predict(X_predict_scaled)

Rounding the **Preducted_Trafikkmengde** values to the nearest integer.

In [27]:
predict_df['Predicted_Trafikkmengde'] = predictions_2023.round().astype(int)

In [28]:
predict_df.head(10)

,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Lufttrykk,Hour,Day,Month,Year,Predicted_Trafikkmengde
0,0.0,0.840000,42.800000,1.480000,994.840000,0,1,1,2023,0
1,0.0,0.583333,54.666667,1.283333,995.033333,1,1,1,2023,1
2,0.0,-0.200000,38.666667,0.733333,995.400000,2,1,1,2023,0
3,0.0,-1.316667,135.000000,0.950000,995.666667,3,1,1,2023,0
4,0.0,0.833333,64.333333,3.266667,995.566667,4,1,1,2023,0
5,0.0,0.783333,114.666667,2.166667,995.550000,5,1,1,2023,2
6,0.0,0.400000,76.666667,1.516667,995.383333,6,1,1,2023,1
7,0.0,-1.866667,141.000000,1.550000,995.333333,7,1,1,2023,42
8,0.0,-2.566667,144.000000,1.383333,995.400000,8,1,1,2023,42
9,0.0,-3.116667,137.666667,1.716667,995.566667,9,1,1,2023,22


In [29]:
predict_df.tail()

,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Lufttrykk,Hour,Day,Month,Year,Predicted_Trafikkmengde
4189,0.0,13.680000,148.400000,2.860000,996.800000,19,30,6,2023,22
4190,0.0,13.283333,141.166667,2.316667,996.583333,20,30,6,2023,22
4191,0.0,13.466667,146.000000,3.250000,996.033333,21,30,6,2023,11
4192,0.0,13.550000,143.000000,2.650000,995.600000,22,30,6,2023,10
4193,0.0,13.616667,138.500000,2.050000,995.300000,23,30,6,2023,8


# Visualize The Predicted Data

In [37]:
monthly_avg = predict_df.groupby('Month')['Predicted_Trafikkmengde'].mean()
# plotly express
fig = ex.bar(monthly_avg, x=monthly_avg.index, y=monthly_avg.values,
             labels={'x': 'Month', 'y': 'Average Traffic'},
             title='Average predicted Traffic by Month')

fig.show()

In [35]:
# Aggregating the data by hour and taking the sum of the predicted traffic
hourly_sum = predict_df.groupby('Hour')['Predicted_Trafikkmengde'].sum()

# Creating the figure with plotly
fig = go.Figure(go.Bar(x=hourly_sum.index, y=hourly_sum.values))

fig.update_layout(
    title='Total Hourly Traffic Prediction for 2023',
    xaxis_title='Hour of the Day',
    yaxis_title='Total Predicted Traffic',
    xaxis=dict(tickvals=list(range(24))),
)

fig.show()

In [36]:
save_df = predict_df[['Year', 'Month', 'Day', 'Hour', 'Predicted_Trafikkmengde']].copy()
save_df['Dato'] = pd.to_datetime(save_df[['Year', 'Month', 'Day']])
save_df.rename(columns={'Hour': 'Tid', 'Predicted_Trafikkmengde': 'Prediksjon'}, inplace=True)
save_df = save_df[['Dato', 'Tid', 'Prediksjon']]
save_df.head()

,Dato,Tid,Prediksjon
0,2023-01-01,0,0
1,2023-01-01,1,1
2,2023-01-01,2,0
3,2023-01-01,3,0
4,2023-01-01,4,0


**Saving the predicted data**

In [33]:
#save_df.to_csv('predictions.csv', index=False)

#with open("trained_decision_tree.pkl", "wb") as file:
#    pickle.dump(dt_best, file)

#with open("standard_scaler.pkl", "wb") as file:
#    pickle.dump(standardscaler, file)